In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import string
import nltk 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
%matplotlib inline 

In [67]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [68]:
df = pd.read_csv("olid-training-v1.0.csv",delimiter="\t")
# df.loc[200:300]

In [4]:
def demoji(text):
        emoji_pattern = re.compile("["
                                  u"\U0001F600-\U0001F64F"
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                  u"\U00002702-\U000027B0"
                                  u"\U000024C2-\U0001F251"
                                  u"\U00010000-\U0010ffff"
                                   "]+", flags=re.UNICODE)
        return(emoji_pattern.sub(r'', text))
                                  
     
                                  
   
 

df[u'tweet'] = df[u'tweet'].astype(str)
df[u'tweet'] = df[u'tweet'].apply(lambda x:demoji(x))
df.to_csv('output.csv',index=False, encoding='utf-8') # save to csv file


In [6]:
for i in range(len(df.subtask_b)):
    if(df.subtask_b[i] != "TIN"):
        df.subtask_c[i] ="NOT"

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
training = df.tweet
labels = df.subtask_c

In [8]:
set(labels)

{'GRP', 'IND', 'NOT', 'OTH'}

In [9]:
for i in range(len(training)):## delete all @USER
    training[i] = training[i].replace("@USER","")

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
# training

In [11]:
for i in range(len(training)):## lowercase
    training[i] = training[i].lower()

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
training = training.apply(lambda x: remove_punct(x))

In [13]:
for i in range(len(training)):## lowercase
    training[i] = training[i].replace("url","")

In [14]:
def tokenization(text):###split all words
    text = re.split('\W+', text)
    return text

training_token = training.apply(lambda x: tokenization(x.lower()))


In [70]:
# training_token

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LeoFishLiao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stopword = set(stopwords.words('english'))

In [18]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
training_stop = training.apply(lambda x: remove_stopwords(x))


In [19]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

In [20]:
countVectorizer = CountVectorizer(analyzer=clean_text)
countVector = countVectorizer.fit_transform(training)

In [21]:
df.tweet = training


In [22]:
training = df.tweet[df.subtask_c!="NOT"]
labels = df.subtask_c[df.subtask_c!="NOT"]

In [23]:
set(labels)

{'GRP', 'IND', 'OTH'}

In [24]:
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

13240 Number of tweets has 19925 words


In [25]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
# count_vect_df.to_csv('vector.csv')

In [26]:
text_clf = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(1,2))),
        ('transform',TfidfTransformer()),
        ('clf', MultinomialNB()),])

In [27]:
text_clf.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x000001C0508C55E8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('transform',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=Fals

In [28]:
from sklearn.model_selection import cross_val_score
# clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(text_clf, training, labels, cv=5)

In [29]:
scores

array([0.625     , 0.625     , 0.63096774, 0.63612903, 0.62403101])

In [30]:
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [31]:
test = pd.read_csv('testset-levelc.csv',delimiter = "\t").tweet

In [32]:
predicted = text_clf.predict(test)

In [33]:
test_label = pd.read_csv('labels-levelc.csv')

In [34]:
test_label = test_label.Label

In [35]:
np.mean(predicted == test_label)

0.49295774647887325

In [36]:
#使用其他分类器-------支持向量机

In [37]:
from sklearn.svm import LinearSVC,SVC


In [38]:
text_clf2 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,4))),
        ('tfidf', TfidfTransformer()),
        ('clf', LinearSVC())])

In [39]:
text_clf2.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x000001C0508C55E8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

In [40]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.625     , 0.625     , 0.63096774, 0.63612903, 0.62403101])

In [41]:
predicted = text_clf2.predict(test)

In [42]:
np.mean(predicted == test_label)

0.6384976525821596

In [43]:
##使用随机森林

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
text_clf3 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,2))),
        ('clf',RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0))])

In [46]:
text_clf3.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x000001C0508C55E8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                ('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=3,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
         

In [47]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.625     , 0.625     , 0.63096774, 0.63612903, 0.62403101])

In [48]:
predicted = text_clf3.predict(test)

In [49]:
np.mean(predicted == test_label)

0.4694835680751174

In [50]:
#使用Xgboost

In [51]:
!pip install xgboost

In [52]:
import xgboost as xgb

In [53]:
text_clf4 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,4))),
        ('clf',xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1))])

In [54]:
text_clf4.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x000001C0508C55E8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                               interaction_constraints=None, learning_rate=0.1,
                               max_delta_step=0, max_depth=7,
                               min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=200,
                               n_jobs=10, nthread=10, num_parallel_tree=1,
           

In [55]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.625     , 0.625     , 0.63096774, 0.63612903, 0.62403101])

In [56]:
predicted = text_clf4.predict(test)

In [57]:
np.mean(predicted == test_label)

0.6619718309859155

In [64]:
pre = pd.Series(predicted,name="predicted")
lab = pd.Series(test_label)
pd.DataFrame([test,lab,pre]).T.loc[200:210]


,tweet,Label,predicted
200,"#BCOG @USER she is cool, confident, cocky, and...",IND,IND
201,@USER oh fuck off 😂😂,IND,IND
202,#LiberalHypocrisy #criminal #democrat #evidenc...,GRP,GRP
203,#DemocratPartyOfDomesticTerrorists #Fascist #A...,GRP,IND
204,#auspol I don't know why he is still in his j...,IND,IND
205,#MAGA Tell me ONE good thing #trump has done a...,IND,IND
206,"#ConfirmKavanugh now, stall tactics are DC cro...",GRP,IND
207,@USER @USER @USER @USER Beats the hell NJ out ...,IND,IND
208,#StopEtchecopar? Fuck you all 🖕🖕🖕🖕🖕 Que florez...,IND,IND
209,"#Antifa are mentally unstable cowards, pretend...",GRP,IND


In [59]:
##使用SDG

In [60]:
text_clf5 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text)),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None))])

In [61]:
text_clf5.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x000001C0508C55E8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=5, n_iter_no_c

In [62]:
predicted = text_clf5.predict(test)

In [63]:
np.mean(predicted == test_label)

0.568075117370892